In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Performance metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

from preprocessing import create_and_preprocess_X_y, create_train_test_set
from model import create_y_pred_baseline, calculate_metric, baseline_metric_score


# Tensorflow
#import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras import layers, Sequential

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import LSTM, Flatten, Dense

2023-09-13 17:51:00.372930: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-13 17:51:00.374397: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 17:51:00.413012: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 17:51:00.414276: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 17:51:01.305301: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# Load and preprocess the data using create_and_preprocess_X_y()
X, y = create_and_preprocess_X_y()

In [3]:
X_train, X_test, y_train, y_test = create_train_test_set(X, y, train_size=0.8)

In [4]:
y_train_pred_baseline, y_test_pred_baseline = create_y_pred_baseline(0.8)

In [5]:
y_test_pred_baseline.shape

(13, 100, 106)

In [6]:
y_test.shape

(13, 100, 106)

In [7]:
np.unique(y_test_pred_baseline, return_counts=True)

(array([0., 1.]), array([134322,   3478]))

In [8]:
np.unique(y_test, return_counts=True)

(array([0., 1.]), array([134695,   3105]))

In [9]:
score = calculate_metric(y_test_pred_baseline, y_test)

In [10]:
score

0.9573222061000588

In [11]:
# Define the LSTM model
model = Sequential([
    layers.Masking(mask_value=-1, input_shape=(X_train.shape[1], X_train.shape[2])),
    layers.LSTM(units=64, activation ='tanh', return_sequences=True),
    layers.LSTM(units=64, activation ='tanh', return_sequences=False),
    layers.RepeatVector(100),
    layers.TimeDistributed(layers.Dense(units=106, activation = "softmax"))
])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 900, 106)          0         
                                                                 
 lstm (LSTM)                 (None, 900, 64)           43776     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 repeat_vector (RepeatVecto  (None, 100, 64)           0         
 r)                                                              
                                                                 
 time_distributed (TimeDist  (None, 100, 106)          6890      
 ributed)                                                        
                                                                 
Total params: 83690 (326.91 KB)
Trainable params: 83690 

In [13]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
es = EarlyStopping(patience=30, restore_best_weights=True)

In [15]:
# Train the model
model.fit(X_train, y_train, epochs=500, batch_size=32,validation_split=0.2,
          callbacks=[es]) 

Epoch 1/500
2/2 [==============================] - 10s 3s/step - loss: 12.1868 - accuracy: 0.0076 - val_loss: 12.8277 - val_accuracy: 0.0230
Epoch 2/500
2/2 [==============================] - 3s 2s/step - loss: 12.0762 - accuracy: 0.0303 - val_loss: 12.6791 - val_accuracy: 0.0230
Epoch 3/500
2/2 [==============================] - 4s 2s/step - loss: 11.8614 - accuracy: 0.0097 - val_loss: 12.2589 - val_accuracy: 0.0250
Epoch 4/500
2/2 [==============================] - 4s 2s/step - loss: 11.3638 - accuracy: 0.0089 - val_loss: 11.8702 - val_accuracy: 0.0250
Epoch 5/500
2/2 [==============================] - 4s 2s/step - loss: 10.9371 - accuracy: 0.0089 - val_loss: 11.6542 - val_accuracy: 0.0200
Epoch 6/500
2/2 [==============================] - 3s 2s/step - loss: 10.6758 - accuracy: 0.0126 - val_loss: 11.5362 - val_accuracy: 0.0250
Epoch 7/500
2/2 [==============================] - 3s 2s/step - loss: 10.5140 - accuracy: 0.0132 - val_loss: 11.4584 - val_accuracy: 0.0150
Epoch 8/500
2/2 [==

In [16]:
# Make predictions test 
y_test_pred = model.predict(X_test)

1/1 [==============================] - 3s 3s/step


In [17]:
model_score = calculate_metric(y_test_pred, y_test)

In [18]:
model_score

0.0010650396514808997

In [19]:
y_test_pred

array([[[0.00044005, 0.00071261, 0.0003492 , ..., 0.00030933,
         0.00038657, 0.00029268],
        [0.00044005, 0.00071261, 0.0003492 , ..., 0.00030933,
         0.00038657, 0.00029268],
        [0.00044005, 0.00071261, 0.0003492 , ..., 0.00030933,
         0.00038657, 0.00029268],
        ...,
        [0.00044005, 0.00071261, 0.0003492 , ..., 0.00030933,
         0.00038657, 0.00029268],
        [0.00044005, 0.00071261, 0.0003492 , ..., 0.00030933,
         0.00038657, 0.00029268],
        [0.00044005, 0.00071261, 0.0003492 , ..., 0.00030933,
         0.00038657, 0.00029268]],

       [[0.00030944, 0.00047777, 0.00022925, ..., 0.00036712,
         0.00020741, 0.00031031],
        [0.00030944, 0.00047777, 0.00022925, ..., 0.00036712,
         0.00020741, 0.00031031],
        [0.00030944, 0.00047777, 0.00022925, ..., 0.00036712,
         0.00020741, 0.00031031],
        ...,
        [0.00030944, 0.00047777, 0.00022925, ..., 0.00036712,
         0.00020741, 0.00031031],
        [0.0

In [20]:
values, counts = np.unique(X, return_counts=True)
print(values, counts)

[0. 1.] [5676123  143277]


In [21]:
counts[1]/counts[0]

0.025242053422732384

In [22]:
X.shape

(61, 900, 106)

In [23]:
y_test_pred.shape

(13, 100, 106)

In [24]:
values_ypred, counts_ypred = np.unique(y_test_pred, return_counts=True)